## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.c

In [3]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [4]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [5]:
os.getcwd()

'/content'

In [6]:
# data 불러오기
data_dir = '/content/drive/MyDrive/0830_TextMining_WordEmbedding/data'

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['- 움직임 재현과 함께 공간의 차이 인지',
 '환경에 적응',
 '- 증강현실 아바타, 다양한 역할 수행할 것으로 예측',
 '국내 연구진에 의해 증강현실 아바타의 발판이 될 신기술이 개발됐다',
 'KAIST는 문화기술대학원 이성희 교수 연구팀이 사용자의 움직임을 재현하면서도 원격 공간의 차이를 인지해 바뀐 환경에 적응할 수 있는 증강현실 아바타 기술을 개발했다고 20일 밝혔다',
 '이성희 교수 연구팀은 사용자의 공간에는 식탁용 의자, 원격 공간에는 일인용 소파를 놓은 뒤 사용자가 식탁용 의자에 앉으면 아바타는 소파에 적합한 동작과 자세로 변형해 자리에 앉는 실험에 성공했다',
 '증강현실 아바타는 사용자를 대신해 가상공간 혹은 원격공간에 존재하며, 사용자의 움직임을 그대로 따라 하고 반영한다',
 '그러나 사용자의 공간과 아바타가 존재하는 공간에 차이가 있기 때문에 사용자의 동작을 아바타에 그대로 적용하면 아바타는 가상, 원격 공간의 물체를 그냥 통과해버리는 등 사실성이 떨어지게 된다',
 '연구팀은 이런 문제를 해결하기 위해 사용자의 공간에는 식탁용 의자, 아바타의 공간에는 일인용 소파를 설치하고, 깊이 기반 카메라를 이용해 환경 정보를 분석했다',
 '또한 각 의자 간 대응점과 공간상 사용자의 움직임 간 대응점을 찾아 이쪽 공간에서의 어느 한 점이 저쪽에서는 어디에 위치한 것인지 확인했다',
 '이후 사용자의 식탁용 의자에 앉는 동작을 분석 결과에 비춰 소파에 앉는 동작으로 변형시켜 아바타의 형상을 생성했다',
 '연구팀은 이 과정에서 자세(pose)가 아닌 동작(motion)의 대응점을 찾는데 주력했다',
 '공간 대응관계를 이용한 증강현실 아바타의 동작 생성 기술은 주어진 공간 안에서 사용자의 움직임이 갖는 의미와 의도를 왜곡시키지 않고, 다른 공간에 있는 증강현실 아바타의 동작으로 재생성할 수 있다',
 '이는 사용자가 증강현실 아바타의 공간 환경을 고려하지 않고, 동작을 해도 자연스러운 아바타의 동작이 가능해짐을 의미한다',
 '연구팀은 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stop words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

Okt()라는 객체를 사용한다.

In [8]:
okt = Okt()

하나씩 확인하면서, 버릴 단어와 필요없는 품사를 선택하자!

In [9]:
ex1 = corpus[10]
ex1

'이후 사용자의 식탁용 의자에 앉는 동작을 분석 결과에 비춰 소파에 앉는 동작으로 변형시켜 아바타의 형상을 생성했다'

In [10]:
okt.pos(ex1, stem  = True, norm = True)

[('이후', 'Noun'),
 ('사용자', 'Noun'),
 ('의', 'Josa'),
 ('식탁', 'Noun'),
 ('용', 'Noun'),
 ('의자', 'Noun'),
 ('에', 'Josa'),
 ('앉다', 'Verb'),
 ('동작', 'Noun'),
 ('을', 'Josa'),
 ('분석', 'Noun'),
 ('결과', 'Noun'),
 ('에', 'Josa'),
 ('비추다', 'Verb'),
 ('소파', 'Noun'),
 ('에', 'Josa'),
 ('앉다', 'Verb'),
 ('동작', 'Noun'),
 ('으로', 'Josa'),
 ('변형', 'Noun'),
 ('시키다', 'Verb'),
 ('아바타', 'Noun'),
 ('의', 'Josa'),
 ('형상', 'Noun'),
 ('을', 'Josa'),
 ('생', 'Noun'),
 ('성하다', 'Adjective')]

## 1-(1). Manually Choose stop_words

In [11]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.


def CustomTokenization(sentence):

    stop_words = ['하다','다', '것', '의하다', '돼다', '수', '등'] # 의미없는 단어만을 담은 리스트
    stop_pos = ['Punctuation', 'Josa', 'Adverb', 'Alpha', 'Suffix'] # 의미없는 품사를 담은 리스트
    result = [] # 추출한 단어 중 의미있는 것들을 담은 리스트

    for word,char in okt.pos(sentence, stem = True, norm = True):
        if char not in stop_pos: # 의미없는 품사 제외
            if word not in stop_words: # 의미없는 단어 제외
                result.append(word)

    return result

In [12]:
# 잘 되는지 확인해보자.

CustomTokenization(corpus[0])

['움직임', '재현', '공간', '차이', '인지']

In [13]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

corpus_tokenized[0:10]

  0%|          | 0/1020 [00:00<?, ?it/s]

[['움직임', '재현', '공간', '차이', '인지'],
 ['환경', '적응'],
 ['증강현실', '아바타', '다양하다', '역할', '수행', '예측'],
 ['국내', '연', '구진', '증강현실', '아바타', '발판', '되다', '신', '기술', '개발'],
 ['늘다',
  '문화',
  '기술',
  '대학원',
  '이성희',
  '교수',
  '연',
  '구',
  '팀',
  '사용자',
  '움직임',
  '재현',
  '원격',
  '공간',
  '차이',
  '인지',
  '바뀌다',
  '환경',
  '적응하다',
  '있다',
  '증강현실',
  '아바타',
  '기술',
  '개발',
  '20일',
  '밝히다'],
 ['이성희',
  '교수',
  '연',
  '구',
  '팀',
  '사용자',
  '공간',
  '식탁',
  '용',
  '의자',
  '원격',
  '공간',
  '일',
  '인용',
  '소파',
  '놓다',
  '뒤',
  '사용자',
  '식탁',
  '용',
  '의자',
  '앉다',
  '아바타',
  '소파',
  '적합하다',
  '동작',
  '자세',
  '변형',
  '자리',
  '앉다',
  '실험',
  '성공하다'],
 ['증강현실',
  '아바타',
  '사용자',
  '대신',
  '가상공간',
  '원',
  '격',
  '공간',
  '존재',
  '사용자',
  '움직임',
  '그대로',
  '따르다',
  '반영'],
 ['그러나',
  '사용자',
  '공간',
  '아바타',
  '존재',
  '공간',
  '차이',
  '있다',
  '때문',
  '사용자',
  '동작',
  '아바타',
  '그대로',
  '적용',
  '아바타',
  '가상',
  '원격',
  '공간',
  '물체',
  '그냥',
  '통과',
  '버리다',
  '사실',
  '떨어지다',
  '되다'],
 ['연',
  '구',
  '팀',
  '이렇다',
  '문제

## 1-(2). Using eliminating duplicated more than 40 times.

처음 10개의 문장을 직접 눈으로 보고 필요없는 단어를 골라냈기에 이후 다른 문장들을 보면, 쓸데없는 단어가 많이 들어가있음을 확인할 수 있다.

따라서 추가로 많이 반복되는 단어를 지우는 방식을 적용해보자.

In [14]:
# 의미없다고 판단되는 전체 데이터에서 몇 번씩 나타나는지 확인해보자.

stop_words = ['하다','다', '것', '의하다', '돼다', '수', '등', # 의미없는 단어를 담은 리스트
              '변형','형상','식탁','아바타'] # 의미없는 단어와 비교하기 위해 의미있는 단어를 추가하였다. 

stop_words_dict = dict(zip(stop_words, [0]*len(stop_words)))

for i in range(len(corpus)):
    for j,k in okt.pos(corpus[i], stem = True, norm = True):
        if j in stop_words:
            stop_words_dict[j] += 1

stop_words_dict

{'하다': 946,
 '다': 47,
 '것': 172,
 '의하다': 13,
 '돼다': 88,
 '수': 188,
 '등': 107,
 '변형': 4,
 '형상': 2,
 '식탁': 4,
 '아바타': 16}

의미있는 단어['변형','형상','식탁','아바타']들의 경우에는 반복되는 횟수가 매우 적었다.

그러나 믜이없는 단어들의 반복횟수를 보았을 때 극단적으로는 40번 이상, 보수적으로는 대략 100번이상 단어가 나오면 그 단어가 의미없다고 판단해도 무방할 것이다.

In [15]:
stop_words = ['하다','다', '것', '의하다', '돼다', '수', '등'] # 의미없는 단어만을 담은 리스트
stop_pos = ['Punctuation', 'Josa', 'Adverb', 'Alpha', 'Suffix'] # 의미없는 품사를 담은 리스트

all_freq = {} # 모든 단어와 그 반복횟수를 나타낸 딕셔너리
stop_words_freq = {} # 전체 단어에서 빈번하게 나타나는 단어와 그 반복횟수를 담은 딕셔너리

stop_words_selectedByFreq = [] # 빈도수가 40번 이상인 단어들을 담은 리스트

for sentence in corpus:
    for word,char in okt.pos(sentence, stem = True, norm = True):
        if char not in stop_pos: # 의미없는 품사 제외
            if word not in stop_words: # 의미없는 단어 제외
                if word not in all_freq.keys(): # 새로 단어가 추가될 때,
                    all_freq[word] = 1
                else: # 이미 단어가 있을 때,
                    all_freq[word] += 1

for word in all_freq.keys():
    if all_freq[word] >= 40: # 40번 이상 나타나는 단어이면
        stop_words_freq[word] = all_freq[word]

stop_words_selectedByFreq = list(stop_words_freq.keys())

In [16]:
stop_words_freq

{'연': 110,
 '되다': 209,
 '기술': 90,
 '개발': 80,
 '교수': 82,
 '구': 64,
 '팀': 69,
 '있다': 313,
 '이용': 53,
 '이': 161,
 '결과': 43,
 '가다': 40,
 '의': 60,
 '않다': 48,
 '이다': 56,
 '“': 82,
 '연구': 120,
 '”': 82,
 '말': 44,
 '‘': 84,
 '1': 47,
 '만들다': 70,
 '사람': 46,
 '없다': 43,
 '고': 65,
 '때': 44,
 '물질': 58,
 '명': 42,
 '지구': 59,
 '통해': 51}

In [17]:
# 앞서 추출한 겹치는 단어를 stop word로 추가하여 다시 Tokenization을 진행해보자.

def transCustomTokenization(sentence):

    stop_words = ['하다','다', '것', '의하다', '돼다', '수', '등'] # 의미없는 단어만을 담은 리스트
    stop_words += stop_words_freq # 40번 이상 겹치는 것을 추가
    stop_pos = ['Punctuation', 'Josa', 'Adverb', 'Alpha', 'Suffix'] # 의미없는 품사를 담은 리스트
    result = [] # 추출한 단어 중 의미있는 것들을 담은 리스트

    for word,char in okt.pos(sentence, stem = True, norm = True):
        if char not in stop_pos: # 의미없는 품사 제외
            if word not in stop_words: # 의미없는 단어 제외
                result.append(word)

    return result

In [18]:
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(transCustomTokenization(i))

corpus_tokenized[10:20]

  0%|          | 0/1020 [00:00<?, ?it/s]

[['이후',
  '사용자',
  '식탁',
  '용',
  '의자',
  '앉다',
  '동작',
  '분석',
  '비추다',
  '소파',
  '앉다',
  '동작',
  '변형',
  '시키다',
  '아바타',
  '형상',
  '생',
  '성하다'],
 ['과정', '자세', '아니다', '동작', '대응', '점', '찾다', '주력'],
 ['공간',
  '대응',
  '관계',
  '증강현실',
  '아바타',
  '동작',
  '생',
  '주어지다',
  '공간',
  '안',
  '사용자',
  '움직임',
  '갖다',
  '의미',
  '의도',
  '왜곡',
  '시키다',
  '다른',
  '공간',
  '증강현실',
  '아바타',
  '동작',
  '재생',
  '성하다'],
 ['사용자',
  '증강현실',
  '아바타',
  '공간',
  '환경',
  '고려',
  '동작',
  '해도',
  '자연스럽다',
  '아바타',
  '동작',
  '가능하다',
  '짐',
  '의미'],
 ['증강현실', '아바타', '멀다', '미래', '우리', '일상', '공간', '공유', '다양하다', '역할', '수행', '예상'],
 ['이번',
  '원격',
  '인간',
  '동작',
  '공존',
  '공간',
  '가구',
  '환경',
  '맞추다',
  '증강현실',
  '아바타',
  '적용',
  '위',
  '핵심'],
 ['한편',
  '이번',
  '지난',
  '9월',
  '19일',
  '부터',
  '5일',
  '간',
  '멕시코',
  '메리다',
  '열리다',
  '국제',
  '학술',
  '대회',
  '증강',
  '및',
  '혼합',
  '현실',
  '학회',
  '2016',
  '발표'],
 ['소',
  '두',
  '증',
  '유발',
  '알려지다',
  '지',
  '카',
  '바이러스',
  '남성',
  '생식능력',
  '저하',
  '시키다',
  '동물',
 

# 2번

1. model = gensim.models.Word2Vec.load('/content/drive/MyDrive/0830_TextMining_WordEmbedding/ko.bin') : 기존에 학습된 모델을 가져온다. 단어와 모델의 weight까지 모두 학습이 자동으로 완료된다.

2. model2 = gensim.models.Word2Vec(size = model.wv.vector_size, min_count = 0) : 단순히 모델을 생성. 학습할 단어는 없기에 학습을 못한다.

3. model2.build_vocab(train_set) : 모델에 단어를 추가하고 이후 자동으로 모델의 학습까지 진행

4. model2 = gensim.models.Word2Vec(sentence=train_set, size = model.wv.vector_size, min_count = 0) : sentence에 단어를 넣어주면, 넣어준 단어에 대해 학습이 진행된다. 즉, 위 2,3을 한 번에 수행한 것과 동일

5. model2.intersect_word2vec_format('/content/drive/MyDrive/0830_TextMining_WordEmbedding/ko.bin.gz', binary = False, lockf = 1.0) : 기존의 학습된 모델을 가져와서 넣는다. 1의 명령어와 달리 기존에 학습된 모델에 추가로 적용이 가능함

6. model2.train(train_set, total_examples = len(train_set), epochs = 3) : 단어를 새로 추가하고 모델 학습을 진행. 거의 2번 문장과 비슷하다.



이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

hyperparameter

* size : embedding vector의 차원 (default = 100)   
* min_count : n번 이하로 나온 단어는 무시 (default =  5)  
* window : 주변 몇 개의 단어로 학습을 진행할 것인지 (default = 5)   
* sg = 0이면 CBoW, 1이면 Skip Gram (default = 1)   
* hs = 0이면 Hierarchical Softmax 사용 x, 1이면 사용 (default = 1)   
* negative : negative sampling 의 noise words 개수 (default = 5)   
* alpha : 초기 learning rate (default = 0.025)   
* min_alpha : 최소 learning_rate   
* seed : random seed   

## 2-(1). model_no_pretrained : 사전학습 없이 모델 학습

모델 학습 순서
1. (선택) 사전학습 모델을 올린다.
model = gensim.models.WordwVec.load("ko.bin")
2. 모델에 사용할 단어를 올린다.
model.build_vocab(쪼갠단어들=tokenization을 마무리한 뒤의 단어들)
3. 모델을 학습시킨다.


In [19]:
# 사전학습 없이 모델 학습
# 모델 생성

# 모델 성능이 너무 별로이고 시간이 너무 짧게 걸려서
# window는 높이고, alpha는 낮추고, hs=0으로 하여 더 성능이 좋고 시간이 걸리게 했다.
model_no_pretrained = Word2Vec(size = 200, min_count = 0, window=7, alpha=0.001, hs=0)

# 모델 vocabulary 추가
# corpus_tokenized : 이전에 만들었던 이중 리스트; 문장에서 쪼갠 단어가 들어있음.
model_no_pretrained.build_vocab(corpus_tokenized)

# 모델에 있는 corpus(문장)의 수
total_examples = model_no_pretrained.corpus_count # <- len(corpus)와 동일!
print("corpus 수 :", total_examples)

# 문장을 tokenization하여 만든 단어의 수
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

corpus 수 : 1020
현재 vocabulary size(V) : 3816


In [20]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

# 아래 warning이 나오는데, 뭐지...?

(131873, 132000)

In [21]:
# 여기서 단어를 선택해 아래에서 모델이 잘 돌아가는지 확인해보자.
corpus_tokenized

[['움직임', '재현', '공간', '차이', '인지'],
 ['환경', '적응'],
 ['증강현실', '아바타', '다양하다', '역할', '수행', '예측'],
 ['국내', '구진', '증강현실', '아바타', '발판', '신'],
 ['늘다',
  '문화',
  '대학원',
  '이성희',
  '사용자',
  '움직임',
  '재현',
  '원격',
  '공간',
  '차이',
  '인지',
  '바뀌다',
  '환경',
  '적응하다',
  '증강현실',
  '아바타',
  '20일',
  '밝히다'],
 ['이성희',
  '사용자',
  '공간',
  '식탁',
  '용',
  '의자',
  '원격',
  '공간',
  '일',
  '인용',
  '소파',
  '놓다',
  '뒤',
  '사용자',
  '식탁',
  '용',
  '의자',
  '앉다',
  '아바타',
  '소파',
  '적합하다',
  '동작',
  '자세',
  '변형',
  '자리',
  '앉다',
  '실험',
  '성공하다'],
 ['증강현실',
  '아바타',
  '사용자',
  '대신',
  '가상공간',
  '원',
  '격',
  '공간',
  '존재',
  '사용자',
  '움직임',
  '그대로',
  '따르다',
  '반영'],
 ['그러나',
  '사용자',
  '공간',
  '아바타',
  '존재',
  '공간',
  '차이',
  '때문',
  '사용자',
  '동작',
  '아바타',
  '그대로',
  '적용',
  '아바타',
  '가상',
  '원격',
  '공간',
  '물체',
  '그냥',
  '통과',
  '버리다',
  '사실',
  '떨어지다'],
 ['이렇다',
  '문제',
  '해결',
  '위해',
  '사용자',
  '공간',
  '식탁',
  '용',
  '의자',
  '아바타',
  '공간',
  '일',
  '인용',
  '소파',
  '설치',
  '깊이',
  '기반',
  '카메라',
  '환경',
  '정보',
  

In [22]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('일본인', topn=5)

[('모으다', 0.2267201542854309),
 ('㎏', 0.22005760669708252),
 ('일련', 0.2197701632976532),
 ('고가', 0.2145441472530365),
 ('보조', 0.20288224518299103)]

In [23]:
model_no_pretrained.wv.most_similar('총리', topn=5)

[('결과물', 0.22787168622016907),
 ('이끌다', 0.22658398747444153),
 ('어디', 0.22189828753471375),
 ('열풍', 0.22076000273227692),
 ('배', 0.21141991019248962)]

In [24]:
model_no_pretrained.wv.most_similar('패션', topn=5)

[('노동시간', 0.22548195719718933),
 ('입력', 0.21032847464084625),
 ('까닭', 0.20077626407146454),
 ('수가', 0.19640180468559265),
 ('내비게이션', 0.1940973997116089)]

In [25]:
model_no_pretrained.wv.most_similar('안경', topn=5)

[('65%', 0.26579686999320984),
 ('시너지', 0.2367364764213562),
 ('휘발', 0.2283756136894226),
 ('메타', 0.2272164225578308),
 ('저공', 0.21325954794883728)]

In [26]:
model_no_pretrained.wv.most_similar('박근혜', topn=5) # 확실히 이상하다...

[('인프라', 0.2621699571609497),
 ('학교', 0.23468106985092163),
 ('두뇌', 0.23242834210395813),
 ('사이즈', 0.23125974833965302),
 ('센서', 0.22335165739059448)]

## 2-(2). pretrained : 사전학습 된 모델을 사용하여 학습

In [27]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('/content/drive/MyDrive/0830_TextMining_WordEmbedding/ko.bin')

total_examples = pretrained.corpus_count
print("corpus 수 :", total_examples)

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지'])) # 즉, 강아지에 대해 200개의 숫자열이 존재

# 모델 생성
model_pretrained = Word2Vec(size = 200, min_count = 0, window=7, alpha=0.001, hs=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

corpus 수 : 2203153
Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3816


In [28]:
# pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

# embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('/content/drive/MyDrive/0830_TextMining_WordEmbedding/ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31501


In [29]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 30)

(395971, 396000)

In [30]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('박근혜', topn=5) # great!

[('김대중', 0.63658607006073),
 ('박정희', 0.6239240765571594),
 ('노무현', 0.6041322946548462),
 ('김영삼', 0.5929562449455261),
 ('윤휴', 0.5835036635398865)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
일단 사전학습이 없으면, 성능이 매우 떨어졌습니다. 그리고 그 이유는 다음과 같을 것 같습니다.

1. 사전학습이 없는 모델은 corpus 수 : 1020, vocabulary size(V) : 3816인 것에 비해, 사전학습을 한 모델은 corpus 수 : 2203153
Pretrained model의 단어 수(V) : 30185라서 압도적인 데이터 양이 있는 것
2. 최적화된 hyper-parameter으로 이미 적합된 모델이 pretrained model이라는 것

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

## 4-(1). 기본 시각화

In [31]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, []))) # 단어를 다 하나씩 펴서 중복을 제거하고 하나의 리스트에 담는다.
len(unique_text)

3816

In [32]:
import numpy as np

np.array(sum(corpus_tokenized, []))

array(['움직임', '재현', '공간', ..., '질병', '도움', '밝히다'], dtype='<U13')

In [33]:
unique_text[:5]

['소용', '5일', '높이', '대립형질', '다루다']

In [34]:
model_no_pretrained.wv['움직임'].shape # 총 200개의 피쳐

(200,)

즉, 3816개의 단어가 200개의 피쳐값을 각각 가지고 있다.

In [37]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i]) # 사전학습된 모델을 사용한다.

In [38]:
np.array(vector).shape

(3816, 200)

In [39]:
np.array(word).shape

(3816,)

단어는 word에, 피쳐값인 벡터는 vector에 집어넣었다.

In [40]:
pca = PCA(n_components=3).fit_transform(vector) # 3차원

# 좀 걸린다.
tsne = TSNE(n_components=3, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector) # 3차원

In [41]:
# 3차원 PCA 시각화

data = go.Scatter3d(
    x=pca[:,0],
    y=pca[:,1],
    z=pca[:,2],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=3,
        # color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


## 4-(2). 색깔을 넣은 시각화

색깔이 같아서 의미있는 관찰이 어렵다. 따라서 유사한 단어들끼리 색을 같은걸로 설정해보자.

In [42]:
# 이것도 좀 걸린다.

color_dict = dict(zip(range(len(word)),[0]*len(word))) # 각 단어별로 유사한 단어를 5개씩 저장해둔다.

for i in range(len(word)):
    result = model_pretrained.wv.most_similar(word[i], topn=5) # 비슷하다고 나온 단어. 상위 5개
    color_dict[i] = list(np.array(result)[:,0]) # 단어만 추출해서 리스트형태로 집어넣는다.
    color_dict[i].append(word[i]) # 자기자신도 넣는다.

In [43]:
len(color_dict) # 총 3816개의 단어에 대해

3816

In [44]:
color_dict[0] # 자기자신을 포함 총 6개의 단어

['틀림', '쓸모', '손색', '가망', '가차', '소용']

In [50]:
# 한 6분 정도 걸린다.

# 이제 단어별로 비슷한 단어를 5개씩 저장했으니, 자기자신까지 하여 총 6개를 각 그룹에 대해
# 그룹별로 원소가 하나라도 겹치면 합치는 작업을 진행하자.

done_group_list = [] # 이미 합쳐진 그룹의 번호
no_overlap = False

temp_dict = color_dict.copy() # iteration 중에 for의 조건에 있는 dictionary가 바뀌면 안됨으로

while(no_overlap==False):
    no_overlap = True
    for key1,value1 in color_dict.items():
        for key2,value2 in color_dict.items():
            if key1 < key2:
                if key1 not in done_group_list and key2 not in done_group_list: # 아직 합쳐지지 않은 그룹들에 대하여

                    if list(np.intersect1d(np.array(value1), np.array(value2))) != []: # value1과 value2에서 겹치는 원소가 있다면
                        done_group_list.append(key2) # 겹친 그룹의 index값을 넣어준다.

                        temp_dict[key1] = list(set(color_dict[key1] + value2)) # 합쳐준다.
                        temp_dict.pop(key2) # key2는 없애준다.
                        
                        no_overlap = False
    color_dict = temp_dict.copy()

In [51]:
color_dict

{0: ['고속버스',
  '트래픽',
  '하드웨어',
  '소용',
  '사업인',
  '인공위성',
  '농민',
  '매각',
  '노선',
  '한도',
  '협궤',
  '로망',
  '약하다',
  '위성',
  '대가',
  '생활비',
  '걸다',
  '룩',
  '노동',
  '칼미크인',
  '60',
  '예습',
  '7,000회',
  '무선',
  '빔',
  '라우터',
  '우주인',
  '지난달',
  '블록',
  '두절',
  '계열사',
  '인수',
  '트램',
  '업종',
  '전자쌍',
  '걸치다',
  '체인',
  '비행사',
  '포밍',
  '네트워크',
  '방도',
  '면역체계',
  '컨트롤',
  '웃다',
  '파장',
  '글러브',
  '한계',
  '소유스',
  '자본가',
  '커리큘럼',
  '클라우드',
  '유닛',
  '실업자',
  '몸값',
  '통신망',
  '컴퓨터',
  '끊기',
  '납품',
  '안전하다',
  '지하철역',
  '1만',
  '듀오',
  '쾌속',
  '지하철',
  '차입',
  '애플리케이션',
  '상호작용',
  '크게',
  '일자리',
  '보즈',
  '퇴사',
  '노트북',
  '거액',
  '다이트',
  '슬롯',
  '솔루션',
  '로켓',
  '가차',
  '석학',
  '가망',
  '모듈',
  '마이크로프로세서',
  '폐업',
  '틀림',
  '공연',
  '도저히',
  '브',
  '비행체',
  '쓸모',
  '기업인',
  '냥',
  '근로자',
  '밴드',
  '영역',
  '우주',
  '범위',
  '범주',
  '답례',
  '나위',
  '모기업',
  '건국대',
  '돈',
  '얻다',
  '그룹',
  '우주선',
  '사령선',
  '손색',
  '매입',
  '커패시터',
  '경전철',
  '심다',
  '판로',
  '회선',
  '효과',
  '졸다',
  '고속',
  '순

In [61]:
# 유사단어 중에 corpus_tokenized에 없는 단어가 있는 것 같다.
count = 0
for key,value in color_dict.items():
    count += len(value)
print('새로 만든 단어 수 :', count)
print('이전에 그래프를 그리기 위해 사용한 단어수 :', len(word))

새로 만든 단어 수 : 6797
이전에 그래프를 그리기 위해 사용한 단어수 : 3816


In [63]:
extended_word = []

for key,value in color_dict.items():
    extended_word += value

print(extended_word[:5])
extended_word = list(set(extended_word)) # 단어겹치는지 확인
print(len(extended_word)) # 역시 하나도 안겹치게 잘 나누어졌다.

['고속버스', '트래픽', '하드웨어', '소용', '사업인']
6797


In [64]:
# 시각화할 단어 리스트 생성
unique_text = extended_word

# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i]) # 사전학습된 모델을 사용한다.

In [77]:
# 바뀐 데이터에 대해 다시 학습

pca = PCA(n_components=3).fit_transform(vector) # 3차원

# 좀 걸린다.
tsne = TSNE(n_components=3, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector) # 3차원

In [67]:
print('pca에 들어간 데이터의 수 :', len(pca[:,0])) # okay 딱 맞다!

pca에 들어간 데이터의 수 : 6797


In [70]:
max = max(color_dict.keys())
max

3792

In [71]:
min = min(color_dict.keys())
min

0

In [72]:
color = [] # color값이 저장되는 리스트
# 0~1 사이의 값이 저장되며, 따라서 color_dict의 index를 스케일해준다.

for i in word:
    for key,value in color_dict.items():
        if i in value:
            temp = (key - min) / (max - min)
            color.append(temp)

print(color[:5])
print(len(color)) # 갯수가 word만큼 나왔다. 이 color는 word와 순서대로 1대1 대응이다.

[0.03164556962025317, 0.0, 0.7188818565400844, 0.0007911392405063291, 0.7745253164556962]
6797


In [73]:
# 3차원 PCA 시각화

data = go.Scatter3d(
    x=pca[:,0],
    y=pca[:,1],
    z=pca[:,2],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=3,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




사실 유사한 단어가 그렇게 많이 겹치지 않아서 눈으로 직접적으로 보이지는 않는다.

In [78]:
# 3차원 T-SNE 시각화

data = go.Scatter3d(
    x=tsne[:,0],
    y=tsne[:,1],
    z=tsne[:,2],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=3,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




## 4-(3). 2차원으로 낮춰 시각화

사실 모여있는 않는 것 같이 보인다. 따라서 2차원으로 낮춰서 시각화를 해보자.

In [79]:
# 바뀐 데이터에 대해 다시 학습

pca = PCA(n_components=2).fit_transform(vector) # 3차원

# 좀 걸린다.
tsne = TSNE(n_components=2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector) # 3차원

In [81]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [82]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




2차원이어도 마찬가지이다...

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

perplexity의

낮을수록 더 좋은 모델을 만들어준다.

참고 : https://wikidocs.net/21697

In [83]:
# t-SNE

# 좀 걸린다.
tsne = TSNE(n_components=3, random_state = 0, perplexity = 10, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector) # 3차원


In [84]:
# T-SNE 3차원 시각화

data = go.Scatter3d(
    x=tsne[:,0],
    y=tsne[:,1],
    z=tsne[:,2],
    text = ['point #{}'.format(i) for i in range(len(word))],
    mode='markers',
    marker=dict(
        size=3,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




달라진지 확인이 불가하다..

# 수고하셨습니다!